In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import warnings
import scikitplot as skplt
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
warnings.filter_warnings('ignore')
%matplotlib inline

In [30]:
def binary_confusion_matrix(y, y_hat, as_pct=False, verbose=True):
    cm = pd.DataFrame(confusion_matrix(y, y_hat), 
                      columns=['(+) actual', '(-) actual'],
                      index=['(+) predicted', '(-) predicted'])
    if as_pct:
        cm = cm / cm.sum().sum()
        
    P = cm['(+) actual'].sum()
    N = cm['(-) actual'].sum()
    total = P + N
    TP = cm.loc['(+) predicted', '(+) actual']
    FP = cm.loc['(+) predicted', '(-) actual']
    TN = cm.loc['(-) predicted', '(-) actual']
    FN = cm.loc['(-) predicted', '(+) actual']
    TPR = TP / (TP + FN)          # recall/sensitivity
    TNR = TN / (TN + FP)   # specificity
    FPR = FP / (FP + TN)   # fall-out
    FNR = FN / (FN + TP)   # miss rate
    PPV = TP / (TP + FP)   # precision
    NPV = TN / (TN + FN)   # neg predictive value
    
    if verbose:
        print('''
        Condition Positive:                        %i
        Condition Negative:                        %i
        Total Observations:                        %i
        
        True Positive:                             %i
        True Negative:                             %i
        False Positive:                            %i
        False Negative                             %i
        
        True Positive Rate (recall):               %.2f%%
        True Negative Rate (specificity):          %.2f%%
        False Positive Rate (fall-out):            %.2f%%
        False Negative Rate (miss rate):           %.2f%%
        
        Positive Predictive Value (precision):     %.2f%%
        Negative Predictive Value:                 %.2f%%
        ''' %(P, N, total,
             TP, TN, FP, FN,
             TPR*100, TNR*100, FPR*100, FNR*100, 
             PPV*100, NPV*100))
        
    metrics = {'P': P, 'N': N, 'total': total, 
              'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN,
              'TPR': TPR, 'TNR': TNR, 'FPR': FPR, 'FNR': FNR, 'PPV': PPV, 'NPV': NPV}
    
    return cm, metrics



class ColumnExtractor(BaseEstimator, TransformerMixin):
    '''
    Transformer for extracting columns in sklearn pipeline
    '''
    def __init__(self, cols):
        self.cols = cols

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xcols = X[self.cols]
        return Xcols
    
class LabelEncodeObjects(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        assert isinstance(data, pd.DataFrame)
        data = data.astype(str)
        data = data.apply(LabelEncoder().fit_transform)
        return data

In [6]:
train_df = pd.read_csv('data/train_data.csv')
train_df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,z217,z218,z219,z220,z221,z222,subject,phase,state,output
0,0,0,0,1,-323.106605,2.694366,-1.987520,261.092784,0.013704,0.000100,...,-0.004930,-0.005554,5.246375,-7.534092,3.530736,-0.539045,K,3,C,1
1,0,0,0,1,-376.084691,0.969696,-6.933765,355.311648,0.030292,-0.000153,...,0.022757,0.052506,-3.727741,-2.854443,-0.699268,-0.054074,A,4,C,1
2,0,0,0,0,91.955425,2.621643,-2.581162,51.357206,0.036668,-0.000104,...,-0.086813,-0.101497,-7.510594,19.564182,-17.008130,4.945392,D,3,C,1
3,0,0,0,1,-391.814586,1.866914,-2.510799,382.900317,0.007947,-0.000028,...,0.030856,-0.161398,-6.435819,2.174453,-0.153956,-0.003958,G,2,C,0
4,0,0,0,0,-363.823732,2.951346,-3.726368,330.527539,0.010074,-0.000004,...,-0.017226,-0.016454,-2.581403,3.011932,-1.281361,0.192647,C,2,C,1


In [7]:
train_df.describe()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,z215,z216,z217,z218,z219,z220,z221,z222,phase,output
count,4584.0,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,...,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000,4584.000000
mean,0.0,0.000218,0.000218,0.625436,-270.199043,1.801323,-4.654634,253.735831,0.021039,-0.000004,...,-0.001628,0.001754,0.001374,0.001404,-57.916190,96.554052,-52.630948,28.735112,2.519634,0.844895
std,0.0,0.014770,0.014770,0.484063,163.971643,1.143173,3.714967,147.405918,0.026384,0.000472,...,0.099405,0.070156,0.078443,0.091211,243.742549,597.912778,599.208382,217.842477,1.158998,0.362044
min,0.0,0.000000,0.000000,0.000000,-474.588020,-0.237454,-19.295360,0.026692,0.005007,-0.004008,...,-3.805788,-0.999448,-1.886137,-1.264762,-2182.646032,-6674.270678,-8311.046315,-3671.954955,1.000000,0.000000
25%,0.0,0.000000,0.000000,0.000000,-383.328439,0.815996,-6.708809,108.809493,0.007504,-0.000081,...,-0.022183,-0.018118,-0.017379,-0.021389,-31.817506,-1.715924,-11.180312,-0.050475,1.000000,1.000000
50%,0.0,0.000000,0.000000,1.000000,-351.317222,1.578367,-3.324030,313.157669,0.011282,0.000003,...,-0.000383,0.000673,0.000374,0.000507,-2.919292,0.098110,-0.492411,0.001721,3.000000,1.000000
75%,0.0,0.000000,0.000000,1.000000,-177.051209,2.741501,-2.027700,370.832095,0.021861,0.000083,...,0.020556,0.020727,0.019355,0.022440,-0.277689,13.684733,-0.032526,0.483496,4.000000,1.000000
max,0.0,1.000000,1.000000,1.000000,418.085156,4.368496,6.250661,588.309282,0.423408,0.003588,...,0.916725,0.708321,1.548114,2.949784,2408.234281,6111.797852,6412.126601,2816.376179,4.000000,1.000000


In [8]:
train_df.shape

(4584, 670)

In [12]:
train_df.isnull().sum().sum()

0

# Fit baseline model with `LogisticRegression`

In [27]:
# specify columns
y_col = ['output']
cat_cols = ['subject', 'phase', 'state']
num_cols = [col for col in train_df.columns if col not in cat_cols+y_col]
X_cols = cat_cols + num_cols

# split data
X, y = train_df[X_cols], train_df[y_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y)


pipeline = Pipeline([
    ('features', FeatureUnion([
        ('continuous', Pipeline([
            ('extract', ColumnExtractor(num_cols)),
            ('scale', StandardScaler())
        ])),
        ('factors', Pipeline([
            ('extract', ColumnExtractor(cat_cols)),
            ('label', LabelEncodeObjects()),
            ('encode', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
    ('clf', LogisticRegression())
])
     
# fit pipeline
pipeline.fit(X_train, y_train)

# get metrics 
y_pred_train, y_pred_test = pipeline.predict(X_train), pipeline.predict(X_test)
val_acc, val_f1 = accuracy_score(y_pred_test, y_test), f1_score(y_pred_test, y_test)
train_acc, train_f1 = accuracy_score(y_pred_train, y_train), f1_score(y_pred_train, y_train)

print('''
Training Accuracy = %.4f
Validation Accuracy = %.4f

Training F1 Score = %.4f
Validation F1 Score = %.4f
''' %(train_acc, val_acc, train_f1, val_f1))

/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training Accuracy = 0.8881
Validation Accuracy = 0.8416

Training F1 Score = 0.9363
Validation F1 Score = 0.9096



In [28]:
cm, metrics = binary_confusion_matrix(y_test, pipeline.predict(X_test))
cm


        Condition Positive:                        127
        Condition Negative:                        1249
        Total Observations:                        1376
        
        True Positive:                             61
        True Negative:                             1097
        False Positive:                            152
        False Negative                             66
        
        True Positive Rate (recall):               48.03%
        True Negative Rate (specificity):          87.83%
        False Positive Rate (fall-out):            12.17%
        False Negative Rate (miss rate):           51.97%
        
        Positive Predictive Value (precision):     28.64%
        Negative Predictive Value:                 94.33%
        


,(+) actual,(-) actual
(+) predicted,61,152
(-) predicted,66,1097


In [ ]:
skplt.metrics.plot_roc_curve(y_test, pipeline.predict_proba(X_test))

# Test `RandomForestClassifier` with `GridSearchCV`

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {'clf__n_estimators': [250, 500, 750],
         'clf__max_depth': [2, 100, None], 
         'clf__max_features': [.25, .50, .75]}

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('continuous', Pipeline([
            ('extract', ColumnExtractor(num_cols)),
            ('scale', StandardScaler())
        ])),
        ('factors', Pipeline([
            ('extract', ColumnExtractor(cat_cols)),
            ('label', LabelEncodeObjects()),
            ('encode', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
    ('clf', RandomForestClassifier())
])
     
# grid search to fit pipeline
grid = GridSearchCV(pipeline, params, n_jobs=-1)
grid.fit(X_train, y_train)

# check best
print('Best Model')
print(grid.best_estimator_)

# get metrics 
y_pred_train, y_pred_test = grid.predict(X_train), grid.predict(X_test)
val_acc, val_f1 = accuracy_score(y_pred_test, y_test), f1_score(y_pred_test, y_test)
train_acc, train_f1 = accuracy_score(y_pred_train, y_train), f1_score(y_pred_train, y_train)

print('''
Training Accuracy = %.4f
Validation Accuracy = %.4f

Training F1 Score = %.4f
Validation F1 Score = %.4f
''' %(train_acc, val_acc, train_f1, val_f1))

/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was 

/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/Users/washburp/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was 

In [ ]:
cm, metrics = binary_confusion_matrix(y_test, y_pred_test)
cm

In [ ]:
skplt.metrics.plot_roc_curve(y_test, pipeline.predict_proba(X_test))